In [21]:
import pandas as pd
import numpy as np
import os
import geocoder

In [69]:
# load data
dbscan_clusters = pd.read_csv('../../data/evaluation_data/dbscan_clusters.csv', index_col=0)
average_house_price = pd.read_csv('../../data/evaluation_data/average_house_price.csv')
median_house_price = pd.read_csv('../../data/evaluation_data/median_house_price.csv')
main_data = pd.read_csv('../../data/old_datasets/main_dataset.csv')

In [111]:
dbscan_clusters

,id,borough,neighborhood,longitude,latitude,population,PC1,PC2,PC3,cluster
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0,-3.719006,0.891275,-0.730744,0
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0,-4.031917,0.652034,0.068637,0
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0,-3.743484,0.822438,-1.386917,0
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0,-4.466648,2.128319,-1.030878,0
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0,-2.387189,0.928239,-0.489059,0
...,...,...,...,...,...,...,...,...,...,...
301,nyu_2451_34572.302,Manhattan,Hudson Yards,-74.000111,40.756658,23383.0,7.879967,-0.532664,-1.791089,2
302,nyu_2451_34572.303,Queens,Hammels,-73.805530,40.587338,12295.0,-3.992846,1.040376,-1.170375,0
303,nyu_2451_34572.304,Queens,Bayswater,-73.765968,40.611322,25029.0,-4.340139,1.840117,-1.349975,0
304,nyu_2451_34572.305,Queens,Queensbridge,-73.945631,40.756091,6677.0,-0.686059,-0.709019,-0.912492,0


In [23]:
# clean average house price

for column in average_house_price.loc[:,'studio':]:
    average_house_price[column] = average_house_price[column].str.replace(',','')
    average_house_price[column] = average_house_price[column].str.replace('$','')
    average_house_price[column] = average_house_price[column].str.replace(' ','')
    average_house_price[column] = average_house_price[column].str.replace('-','0')
    average_house_price[column] = average_house_price[column].astype(int)

/var/folders/q4/hl7yv5yn6g10s9npc_7kk08h0000gn/T/ipykernel_884/1913209532.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  average_house_price[column] = average_house_price[column].str.replace('$','')


In [24]:
# add mean of house types column
average_house_price['average_price'] = average_house_price.loc[:,'studio':].mean(axis=1)

In [25]:
# drop other columns
average_house_price = average_house_price.drop(['studio', '1_bedroom', '2_bedroom', '3_bedroom'], axis=1)

In [26]:
median_house_price = median_house_price.loc[:,:'type']

In [27]:
median_house_price['average price']

0      $1,975
1      $1,650
2      $1,731
3      $1,898
4      $2,491
        ...  
531    $1,983
532    $1,400
533    $5,345
534    $1,800
535    $1,377
Name: average price, Length: 536, dtype: object

In [28]:
# clean median prices

for value in median_house_price:
    print(value)
    median_house_price[value] = median_house_price[value].str.replace(',','')
    median_house_price[value] = median_house_price[value].str.replace('$','')
    median_house_price[value] = median_house_price[value].str.replace(' ','')
    median_house_price[value] = median_house_price[value].str.replace('-','0')

neighborhood
average price
type


/var/folders/q4/hl7yv5yn6g10s9npc_7kk08h0000gn/T/ipykernel_884/4237213198.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  median_house_price[value] = median_house_price[value].str.replace('$','')


In [29]:
median_house_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   neighborhood   536 non-null    object
 1   average price  536 non-null    object
 2   type           536 non-null    object
dtypes: object(3)
memory usage: 12.7+ KB


In [30]:
median_house_price['average price'] = median_house_price['average price'].astype(int)

In [31]:
median_house_price = median_house_price.groupby('neighborhood').mean().reset_index()

In [32]:
median_house_price = pd.DataFrame(median_house_price)

In [33]:
median_house_price

,neighborhood,average price
0,Allerton,1625.000000
1,Annadale,1450.000000
2,ArdenHeights,2250.000000
3,Astoria,1883.333333
4,Auburndale,1916.666667
...,...,...
211,Woodhaven,1702.333333
212,WoodlawnHeights,1900.000000
213,Woodrow,1400.000000
214,Woodside,1807.666667


In [34]:
average_house_price

,borough,neighborhood,average_price
0,manhatan,Chelsea,6221.00
1,manhatan,East Village,3805.25
2,manhatan,Flatiron/Union Square,8128.25
3,manhatan,Gramercy Park,4716.00
4,manhatan,Greenwich Village,6840.50
5,manhatan,Battery Park City,5957.25
6,manhatan,Financial District,5909.50
7,manhatan,Lower East Side,4119.00
8,manhatan,NoHo,9332.25
9,manhatan,NoLiTa/Little Italy,5611.00


In [35]:
# remove borough
average_house_price = average_house_price.drop('borough', axis=1)

In [36]:
# let's merge the two
house_costs = pd.concat([average_house_price.set_index('neighborhood'), median_house_price.set_index('neighborhood')], join='outer', axis=1).reset_index()

In [95]:
# stack on neighborhood
house_costs = pd.read_csv('house_costs_w.csv', index_col=0)

In [96]:
house_costs

,neighborhood,average_price
0,Chelsea,5712
1,East Village,3805
2,Flatiron/Union Square,8128
3,Gramercy Park,4716
4,Greenwich Village,6841
...,...,...
256,Woodhaven,1702
257,WoodlawnHeights,1900
258,Woodrow,1400
259,Woodside,1808


In [38]:
# There are 160 missing values - :( solution could be to geocode
missing = []
for hood in list(house_costs['neighborhood']):
    if hood not in list(dbscan_clusters['neighborhood']):
        missing.append(hood)
missing

['Flatiron/Union Square',
 'Gramercy Park',
 'NoHo',
 'NoLiTa/Little Italy',
 'SoHo',
 'Beekman/Sutton Place',
 'Midtown East',
 'Midtown West',
 'Turtle Bay/United Nations',
 'Park/Fifth Ave. to 79th St.',
 'Broadway Corridor',
 'Central Park West',
 'Lincoln Center',
 'Riverside Dr./West End Ave.',
 'Harlem',
 'Bedford-Stuyvesant',
 'Downtown Brooklyn',
 'DUMBO',
 'South Slope - Greenwood Heights',
 'ArdenHeights',
 'BathBeach',
 'BatteryParkCity',
 'BayRidge',
 'Bedford0Stuyvesant',
 'BedfordPark',
 'BergenBeach',
 'BoerumHill',
 'BoroughPark',
 'Bowery',
 'BrightonBeach',
 'BronxPark',
 'BrooklynHeights',
 'BrooklynNavyYard',
 'BullsHead',
 'CarrollGardens',
 'CastleHill',
 'CastletonCorners',
 'CentralHarlem',
 'CentralPark',
 'CityIsland',
 'CivicCenter',
 'ClasonPoint',
 'Clearview',
 'ClintonHill',
 'Co0OpCity',
 'CobbleHill',
 'CollegePoint',
 'ConcourseVillage',
 'ConeyIsland',
 'CountryClub',
 'CrotonaParkEast',
 'CrownHeights',
 'CypressHills',
 'DonganHills',
 'DowntownBro

In [41]:
# set up variables for geocoding
cluster_hoods = list(dbscan_clusters['neighborhood'])
price_hoods = list(house_costs['neighborhood'])
mapquest_key = os.environ["MAPQUEST_API_KEY"]

In [44]:
price_hoods

['Chelsea',
 'East Village',
 'Flatiron/Union Square',
 'Gramercy Park',
 'Greenwich Village',
 'Battery Park City',
 'Financial District',
 'Lower East Side',
 'NoHo',
 'NoLiTa/Little Italy',
 'SoHo',
 'Tribeca',
 'West Village',
 'Beekman/Sutton Place',
 'Midtown East',
 'Midtown West',
 'Murray Hill',
 'Turtle Bay/United Nations',
 'Carnegie Hill',
 'Lenox Hill',
 'Park/Fifth Ave. to 79th St.',
 'Roosevelt Island',
 'Yorkville',
 'Broadway Corridor',
 'Central Park West',
 'Lincoln Center',
 'Morningside Heights',
 'Riverside Dr./West End Ave.',
 'East Harlem',
 'Hamilton Heights',
 'Harlem',
 'Inwood',
 'Washington Heights',
 'Bedford-Stuyvesant',
 'Boerum Hill',
 'Brooklyn Heights',
 'Bushwick',
 'Carroll Gardens',
 'Clinton Hill',
 'Cobble Hill',
 'Crown Heights',
 'Downtown Brooklyn',
 'DUMBO',
 'Fort Greene',
 'Gowanus',
 'Greenpoint',
 'Kensington',
 'Park Slope',
 'Prospect Heights',
 'Prospect Lefferts Gardens',
 'Red Hook',
 'South Slope - Greenwood Heights',
 'Sunset Park'

In [50]:
# geocode neighborhood price names
costs_geo = geocoder.mapquest(price_hoods[0]+', New York, USA', key=mapquest_key)
costs_geo.latlng

[41.5536, -73.967201]

In [54]:
# geocoder, get latlng for each place, using mapquest API
place_costs_dict = {}
for place in price_hoods:
    print(place)
    response = geocoder.mapquest(place + ' New York, USA')
    place_costs_dict[place] = response.latlng


Chelsea
East Village
Flatiron/Union Square
Gramercy Park
Greenwich Village
Battery Park City
Financial District
Lower East Side
NoHo
NoLiTa/Little Italy
SoHo
Tribeca
West Village
Beekman/Sutton Place
Midtown East
Midtown West
Murray Hill
Turtle Bay/United Nations
Carnegie Hill
Lenox Hill
Park/Fifth Ave. to 79th St.
Roosevelt Island
Yorkville
Broadway Corridor
Central Park West
Lincoln Center
Morningside Heights
Riverside Dr./West End Ave.
East Harlem
Hamilton Heights
Harlem
Inwood
Washington Heights
Bedford-Stuyvesant
Boerum Hill
Brooklyn Heights
Bushwick
Carroll Gardens
Clinton Hill
Cobble Hill
Crown Heights
Downtown Brooklyn
DUMBO
Fort Greene
Gowanus
Greenpoint
Kensington
Park Slope
Prospect Heights
Prospect Lefferts Gardens
Red Hook
South Slope - Greenwood Heights
Sunset Park
Williamsburg
Windsor Terrace
Allerton
Annadale
ArdenHeights
Astoria
Auburndale
BathBeach
BatteryParkCity
BayRidge
Bayside
Bedford0Stuyvesant
BedfordPark
Bellerose
Belmont
Bensonhurst
BergenBeach
Blissville
Boer

In [67]:
# store in df
geo_costs = pd.DataFrame(place_costs_dict).T
geo_costs.reset_index(inplace=True)
geo_costs.columns = ['price_hood','price_lat', 'price_long']

In [101]:
geo_costs = pd.concat([geo_costs,house_costs['average_price']], axis=1)

In [102]:
geo_costs

,price_hood,price_lat,price_long,average_price
0,Chelsea,41.553600,-73.967201,5712
1,East Village,40.824745,-72.718247,3805
2,Flatiron/Union Square,40.735360,-73.989970,8128
3,Gramercy Park,40.561100,-74.170303,4716
4,Greenwich Village,40.728435,-74.002917,6841
...,...,...,...,...
256,Woodhaven,40.689201,-73.858299,1702
257,WoodlawnHeights,43.010667,-77.815831,1900
258,Woodrow,40.543301,-74.198097,1400
259,Woodside,42.521702,-78.765297,1808


In [84]:
# grab geo data and rearrange
cluster_geo = main_data.loc[:,'neighborhood':'latitude']
cluster_geo.columns= ['cluster_hood', 'cluster_long', 'cluster_lat']

# reorder
columns = cluster_geo.columns
columns = [columns[0],columns[2],columns[1]]
cluster_geo = cluster_geo[columns]
cluster_geo

,cluster_hood,cluster_lat,cluster_long
0,Wakefield,40.894705,-73.847201
1,Co-op City,40.874294,-73.829939
2,Eastchester,40.887556,-73.827806
3,Fieldston,40.895437,-73.905643
4,Riverdale,40.890834,-73.912585
...,...,...,...
301,Hudson Yards,40.756658,-74.000111
302,Hammels,40.587338,-73.805530
303,Bayswater,40.611322,-73.765968
304,Queensbridge,40.756091,-73.945631


In [107]:
# define formulas for closest points
matched_prices = []
# for every cluster neighborhood
for row in cluster_geo.itertuples():
    temp_dict = {}
    # for every price neighborhood
    for pdata in geo_costs.itertuples():
        # store absolute difference in lat and lon
        temp_dict[pdata.Index] = abs((row.cluster_lat - pdata.price_lat)) + abs((row.cluster_long - pdata.price_long))
    # append key with minimum value
    matched_prices.append(geo_costs.iloc[min(temp_dict, key=temp_dict.get)]['average_price'])
matched_prices


[1625,
 1900,
 1625,
 1942,
 1942,
 1799,
 1799,
 1625,
 1625,
 1900,
 1717,
 1717,
 1975,
 1679,
 1648,
 1675,
 1648,
 1800,
 1600,
 1742,
 1848,
 2232,
 2232,
 1377,
 1425,
 1848,
 1833,
 1608,
 2326,
 2326,
 1996,
 1996,
 2038,
 1717,
 1800,
 1799,
 1942,
 1717,
 2326,
 2326,
 1608,
 1900,
 1717,
 2215,
 1608,
 1625,
 2604,
 2498,
 1966,
 2985,
 3079,
 1883,
 1812,
 1875,
 2590,
 1963,
 2125,
 1724,
 3460,
 4110,
 1817,
 3138,
 2250,
 2542,
 4064,
 3462,
 3462,
 3462,
 3462,
 2567,
 3460,
 1947,
 1773,
 1773,
 1817,
 1933,
 1933,
 1812,
 1883,
 2498,
 1724,
 2604,
 1812,
 1933,
 4110,
 2133,
 3556,
 2567,
 2590,
 2250,
 1702,
 1817,
 1875,
 2590,
 1933,
 3138,
 3138,
 3138,
 3079,
 2604,
 2783,
 2325,
 1675,
 2468,
 2468,
 2879,
 2879,
 4813,
 3952,
 4813,
 3952,
 6914,
 6825,
 6825,
 2504,
 6272,
 4280,
 6466,
 1650,
 6841,
 6787,
 4119,
 6316,
 2783,
 7869,
 6466,
 6914,
 6914,
 8128,
 4248,
 5910,
 1883,
 1866,
 1866,
 1866,
 1742,
 1683,
 1941,
 1783,
 1783,
 1833,
 2590,
 3210,

In [113]:
cluster_costs = cluster_geo
cluster_costs['average_costs'] = matched_prices
cluster_costs

,cluster_hood,cluster_lat,cluster_long,average_costs
0,Wakefield,40.894705,-73.847201,1625
1,Co-op City,40.874294,-73.829939,1900
2,Eastchester,40.887556,-73.827806,1625
3,Fieldston,40.895437,-73.905643,1942
4,Riverdale,40.890834,-73.912585,1942
...,...,...,...,...
301,Hudson Yards,40.756658,-74.000111,1873
302,Hammels,40.587338,-73.805530,1919
303,Bayswater,40.611322,-73.765968,1919
304,Queensbridge,40.756091,-73.945631,3952


In [114]:
# add id back
cluster_costs = pd.concat([dbscan_clusters['id'],cluster_costs], axis=1)
cluster_costs

,id,cluster_hood,cluster_lat,cluster_long,average_costs
0,nyu_2451_34572.1,Wakefield,40.894705,-73.847201,1625
1,nyu_2451_34572.2,Co-op City,40.874294,-73.829939,1900
2,nyu_2451_34572.3,Eastchester,40.887556,-73.827806,1625
3,nyu_2451_34572.4,Fieldston,40.895437,-73.905643,1942
4,nyu_2451_34572.5,Riverdale,40.890834,-73.912585,1942
...,...,...,...,...,...
301,nyu_2451_34572.302,Hudson Yards,40.756658,-74.000111,1873
302,nyu_2451_34572.303,Hammels,40.587338,-73.805530,1919
303,nyu_2451_34572.304,Bayswater,40.611322,-73.765968,1919
304,nyu_2451_34572.305,Queensbridge,40.756091,-73.945631,3952


In [116]:
# export
cluster_costs.to_csv('../../data/evaluation_data/cluster_housing_costs.csv')